# From dbt Schema to GDPR-Compliant Pipeline — in 5 Minutes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LineageLogic/LakeLogic/blob/main/examples/01_quickstart/03_dbt_pii_quality.ipynb)
[![GitHub](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/LineageLogic/LakeLogic)

## The Scenario

You have a **dbt project** for your `customers` model — schema defined, tests written,
PII columns annotated. Now you need:

- ✅ **Quality enforcement** — run the same checks that live in `schema.yml` at ingestion time
- 🔒 **GDPR compliance** — honour erasure requests and mask PII for dev/test environments  
- 🧪 **Test data** — generate realistic synthetic customers that obey the same contract

Normally these would be three separate tools. With LakeLogic + `from_dbt()`, they all
share the same schema file you already maintain.

---

## What we cover

| Step | What happens |
|---|---|
| **1** | Import `dbt_schema.yml` → LakeLogic contract (inspect what was mapped) |
| **2** | Generate synthetic customers from the dbt schema |
| **3** | Run quality validation — see quarantined rows |
| **4** | GDPR: erase a specific customer (nullify / hash / redact strategies) |
| **5** | GDPR: mask all PII for a dev/test copy |
| **6** | Batch erasure + audit report |
| **7** | CLI reference |


## Setup

In [1]:
from pathlib import Path
import polars as pl

from lakelogic import DataProcessor, DataGenerator, DbtAdapter
from lakelogic.core.gdpr import forget_subjects, mask_pii_columns, generate_erasure_report

# ── Path resolver — works from repo root, examples/, or Colab ──────────────
def find_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        for candidate in [
            base / name,
            base / "examples" / "01_quickstart",
            base / "lakelogic" / "examples" / "01_quickstart",
        ]:
            if candidate.exists():
                return candidate
    return cwd

EXAMPLE_DIR = find_example_dir("01_quickstart")
SCHEMA_FILE = EXAMPLE_DIR / "dbt_schema.yml"

print(f"Schema file : {SCHEMA_FILE}")
print(f"Exists      : {SCHEMA_FILE.exists()}")

Schema file : d:\Github\_SaaS\lakelogic\examples\01_quickstart\dbt_schema.yml
Exists      : True


---

## Step 1 — Import dbt Schema → LakeLogic Contract

`DbtAdapter` reads the dbt YAML and maps:
- `meta: {pii: true}` → `field.pii = True`
- `not_null` → `field.required = True`
- `unique` → dataset-level uniqueness rule (`SELECT COUNT(*) - COUNT(DISTINCT ...) FROM ...`)
- `accepted_values` → `row_rule` IN check
- `expression_is_true` → `row_rule` raw SQL

**No manual YAML editing. No new files to maintain.**

In [2]:
adapter  = DbtAdapter(SCHEMA_FILE)
contract = adapter.model_to_contract("customers")

# ── What got mapped? ────────────────────────────────────────────────
fields     = contract.model.fields if contract.model else []
pii_fields = [f.name for f in fields if f.pii]
req_fields = [f.name for f in fields if f.required]
row_rules  = contract.quality.row_rules    if contract.quality else []
ds_rules   = contract.quality.dataset_rules if contract.quality else []

print(f"Contract    : {contract.dataset}")
print(f"Fields      : {len(fields)}")
print(f"Required    : {req_fields}")
print(f"PII fields  : {pii_fields}  ← auto-detected from meta.pii")
print(f"Row rules   : {len(row_rules)}")
print(f"Dataset rules: {len(ds_rules)}")
print()
for r in row_rules:
    print(f"  row rule  ▸ {r.name}: {r.sql}")
for r in ds_rules:
    print(f"  dset rule ▸ {r.name}: {r.sql}")

Contract    : customers
Fields      : 10
Required    : ['customer_id', 'email', 'full_name', 'country', 'tier', 'total_spend', 'created_at']
PII fields  : ['email', 'full_name', 'phone', 'date_of_birth']  ← auto-detected from meta.pii
Row rules   : 4
Dataset rules: 1

  row rule  ▸ expr_email_email_LIKE: email LIKE '%@%'
  row rule  ▸ country_accepted_values: country IN ('GB', 'DE', 'FR', 'NL', 'SE', 'NO', 'DK', 'FI', 'ES', 'IT', 'US', 'CA', 'AU', 'NZ', 'IN', 'SG', 'JP')
  row rule  ▸ tier_accepted_values: tier IN ('free', 'starter', 'growth', 'enterprise')
  row rule  ▸ expr_total_spend_total_spend_0: total_spend >= 0
  dset rule ▸ customer_id_unique: SELECT COUNT(*) - COUNT(DISTINCT customer_id) FROM customers


### Preview the generated LakeLogic contract YAML

You can also export it to disk — once, if you want to add LakeLogic-specific extras
(streaming config, materialization targets, SLOs) on top of what dbt provides.

In [3]:
# Preview the YAML that would be written
yaml_text = adapter.to_yaml("customers")
print(yaml_text[:2000])  # first 2000 chars

# To write it → contracts/customers.yaml:
#   adapter.to_yaml("customers", output=EXAMPLE_DIR / "contracts" / "customers.yaml")
#
# Or via CLI:
#   lakelogic import-dbt --schema dbt_schema.yml --model customers --output contracts/ --dry-run

version: 1.0.0
info:
  title: customers
  version: 1.0.0
  description: 'Silver-layer customer profiles sourced from the CRM. Contains PII
    and is subject to GDPR right-to-erasure obligations.

    '
  target_layer: silver
metadata: {}
environments: {}
links: []
dataset: customers
primary_key:
- customer_id
lineage:
  enabled: false
  capture_source_path: true
  capture_timestamp: true
  capture_run_id: true
  source_column_name: _lakelogic_source
  timestamp_column_name: _lakelogic_processed_at
  run_id_column_name: _lakelogic_run_id
  capture_contract_name: false
  contract_name_column_name: _lakelogic_contract_name
  capture_domain: true
  capture_system: true
  domain_column_name: _lakelogic_domain
  system_column_name: _lakelogic_system
  preserve_upstream: []
  upstream_prefix: _upstream
  run_id_source: run_id
materialization:
  strategy: append
  partition_by: []
  cluster_by: []
  reprocess_policy: overwrite_partition
  soft_delete_value: true
upstream: []
model:
  fields:


---

## Step 2 — Generate Synthetic Customers

`DataGenerator.from_dbt()` reads the same schema file and generates rows that:
- Respect field types (boolean stays boolean, integer stays integer)
- Honour `accepted_values` — `country` and `tier` are always from the allowed set
- Honour `expression_is_true` — `total_spend` is always ≥ 0 for valid rows
- Inject a controlled ratio of **intentionally invalid rows** to test quarantine

No data file needed. Unblock your pipeline on day one.

In [4]:
gen = DataGenerator.from_dbt(SCHEMA_FILE, model="customers", seed=42)

# 200 rows total: 90% valid, 10% intentionally bad (to exercise quarantine)
df = gen.generate(rows=200, invalid_ratio=0.10)

print(f"Generated  : {len(df)} rows")
print(f"Schema     : {df.schema}")
print()
print(df.head(6))

Generated  : 200 rows
Schema     : Schema({'customer_id': Int64, 'email': String, 'full_name': String, 'phone': String, 'date_of_birth': String, 'country': String, 'tier': String, 'total_spend': Float64, 'created_at': String, 'is_active': Boolean})

shape: (6, 10)
┌────────────┬───────────┬───────────┬───────────┬───┬─────────┬───────────┬───────────┬───────────┐
│ customer_i ┆ email     ┆ full_name ┆ phone     ┆ … ┆ tier    ┆ total_spe ┆ created_a ┆ is_active │
│ d          ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---     ┆ nd        ┆ t         ┆ ---       │
│ ---        ┆ str       ┆ str       ┆ str       ┆   ┆ str     ┆ ---       ┆ ---       ┆ bool      │
│ i64        ┆           ┆           ┆           ┆   ┆         ┆ f64       ┆ str       ┆           │
╞════════════╪═══════════╪═══════════╪═══════════╪═══╪═════════╪═══════════╪═══════════╪═══════════╡
│ -670       ┆ zlgghcip@ ┆ lvgnp     ┆ +19165108 ┆ … ┆ growth  ┆ 688.6     ┆           ┆ false     │
│            ┆ tdjw.io   ┆  

In [5]:
# Verify field rules were injected from dbt tests
fr = gen._build_field_rules()
print(f"country accepted values : {fr.get('country', {}).get('accepted_values')}")
print(f"tier    accepted values : {fr.get('tier', {}).get('accepted_values')}")
print(f"total_spend min         : {fr.get('total_spend', {}).get('min')}")
print()

# Spot-check correct types and valid constrained values
print("Sample PII fields:")
print(df.select(["customer_id","email","full_name","tier","country"]).head(4))

country accepted values : ['GB', 'DE', 'FR', 'NL', 'SE', 'NO', 'DK', 'FI', 'ES', 'IT', 'US', 'CA', 'AU', 'NZ', 'IN', 'SG', 'JP']
tier    accepted values : ['free', 'starter', 'growth', 'enterprise']
total_spend min         : 0.0

Sample PII fields:
shape: (4, 5)
┌─────────────┬───────────────────────┬─────────────┬─────────┬──────────────┐
│ customer_id ┆ email                 ┆ full_name   ┆ tier    ┆ country      │
│ ---         ┆ ---                   ┆ ---         ┆ ---     ┆ ---          │
│ i64         ┆ str                   ┆ str         ┆ str     ┆ str          │
╞═════════════╪═══════════════════════╪═════════════╪═════════╪══════════════╡
│ -670        ┆ zlgghcip@tdjw.io      ┆ lvgnp       ┆ growth  ┆ INVALID_DRUS │
│ 8571        ┆ hbbbclc@svjptatm.org  ┆ bozl01q3m73 ┆ growth  ┆ SG           │
│ 8626        ┆ axfdsxv@bdsdc.co      ┆ 8n7odkef82  ┆ free    ┆ NO           │
│ 8822        ┆ strjbriiw@xnrisoer.io ┆ wktakpu     ┆ starter ┆ IT           │
└─────────────┴───────────

---

## Step 3 — Quality Validation

`DataProcessor.from_dbt()` uses the same schema file and runs all the quality rules
that were mapped from dbt tests — at ingestion time, before the data reaches Silver.

The result splits cleanly into **good** (passes all rules) and **quarantined** (fails any rule).

> ℹ️ With `invalid_ratio=0.10` you should see ~90% good, ~10% quarantined.

In [6]:
proc = DataProcessor.from_dbt(SCHEMA_FILE, model="customers")

good_df, bad_df = proc.run(df)

n_total = len(df)
n_good  = len(good_df) if good_df is not None else 0
n_bad   = len(bad_df)  if bad_df  is not None else 0

print(f"Input rows      : {n_total}")
print(f"Passed (good)   : {n_good}  ({n_good/n_total:.1%})")
print(f"Quarantined     : {n_bad}   ({n_bad/n_total:.1%})")

# Rules that were active:
print()
print("Rules that were active:")
q = proc.contract.quality
if q:
    for r in (q.row_rules or []):
        print(f"  [row]  {r.name}: {r.sql}")
    for r in (q.dataset_rules or []):
        print(f"  [data] {r.name}: {r.sql}")

2026-02-21 13:01:20.555 | INFO     | lakelogic.engines.polars:_run_dataset_rules:391 - Quality Check: customer_id_unique | Result: 1 | Status: FAIL
2026-02-21 13:01:20.568 | INFO     | lakelogic.core.processor:run:410 - Run complete. Source: 200, Total (post-transform): 200, Good: 181, Quarantined: 19, Pre-Transform Dropped: 0, Ratio: 9.50%


Input rows      : 200
Passed (good)   : 181  (90.5%)
Quarantined     : 19   (9.5%)

Rules that were active:
  [row]  expr_email_email_LIKE: email LIKE '%@%'
  [row]  country_accepted_values: country IN ('GB', 'DE', 'FR', 'NL', 'SE', 'NO', 'DK', 'FI', 'ES', 'IT', 'US', 'CA', 'AU', 'NZ', 'IN', 'SG', 'JP')
  [row]  tier_accepted_values: tier IN ('free', 'starter', 'growth', 'enterprise')
  [row]  expr_total_spend_total_spend_0: total_spend >= 0
  [data] customer_id_unique: SELECT COUNT(*) - COUNT(DISTINCT customer_id) FROM customers


In [7]:
# Peek at why bad rows were quarantined
# _lakelogic_errors holds the list of rule names that each row failed
if bad_df is not None and len(bad_df) > 0:
    print("Sample quarantined rows:")
    diag_cols = [c for c in ["customer_id","email","tier","country","total_spend",
                              "_lakelogic_errors"] if c in bad_df.columns]
    print(bad_df.select(diag_cols).head(5))
else:
    print("No bad rows — all invalid rows may have been caught by required-field rules only.")

Sample quarantined rows:
shape: (5, 6)
┌─────────────┬────────────────────┬──────────────┬──────────────┬─────────────┬───────────────────┐
│ customer_id ┆ email              ┆ tier         ┆ country      ┆ total_spend ┆ _lakelogic_errors │
│ ---         ┆ ---                ┆ ---          ┆ ---          ┆ ---         ┆ ---               │
│ i64         ┆ str                ┆ str          ┆ str          ┆ f64         ┆ list[str]         │
╞═════════════╪════════════════════╪══════════════╪══════════════╪═════════════╪═══════════════════╡
│ -670        ┆ zlgghcip@tdjw.io   ┆ growth       ┆ INVALID_DRUS ┆ 688.6       ┆ ["Rule failed:    │
│             ┆                    ┆              ┆              ┆             ┆ created_at_requ…  │
│ null        ┆                    ┆              ┆ JP           ┆ 572.6306    ┆ ["Rule failed:    │
│             ┆                    ┆              ┆              ┆             ┆ customer_id_req…  │
│ 2795        ┆ chnkctrd@esxnl.com ┆ INVALID_IQQY ┆ 

---

## Step 4 — GDPR: Right-to-Erasure

A customer contacts support: *"Please delete all my personal data."*

`forget_subjects()` knows which columns are PII **because the dbt schema said so**
via `meta: {pii: true}`. No extra config. Three strategies available:

| Strategy | What happens to PII values | Referential integrity |
|---|---|---|
| `nullify` | Replaced with `null` | Lost |
| `hash` | SHA-256, deterministic | Preserved — can still JOIN on hashed values |
| `redact` | Replaced with `***REDACTED***` | Lost (human-readable audit trail) |

> **Note**: `forget_subjects` casts `customer_id` to string internally for matching,
> so passing `subject_id` as a string works even though the column is `Int64`.

In [8]:
# Pick a real customer_id from the good dataset to erase
subject_id = str(good_df["customer_id"][0])
print(f"Erasing customer_id = {subject_id}")
print()

# Show before
before = good_df.filter(pl.col("customer_id").cast(pl.Utf8) == subject_id)
print("Before erasure:")
print(before.select(["customer_id","email","full_name","phone"]))
print()

Erasing customer_id = 8571

Before erasure:
shape: (1, 4)
┌─────────────┬──────────────────────┬─────────────┬──────────────┐
│ customer_id ┆ email                ┆ full_name   ┆ phone        │
│ ---         ┆ ---                  ┆ ---         ┆ ---          │
│ i64         ┆ str                  ┆ str         ┆ str          │
╞═════════════╪══════════════════════╪═════════════╪══════════════╡
│ 8571        ┆ hbbbclc@svjptatm.org ┆ bozl01q3m73 ┆ +10926403064 │
└─────────────┴──────────────────────┴─────────────┴──────────────┘



In [9]:
# Strategy 1: NULLIFY — PII fields become null
df_nullified = forget_subjects(
    good_df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=[subject_id],
    erasure_strategy="nullify",
)
after_null = df_nullified.filter(pl.col("customer_id").cast(pl.Utf8) == subject_id)
print("After NULLIFY:")
print(after_null.select(["customer_id","email","full_name","phone"]))

2026-02-21 13:01:21.387 | INFO     | lakelogic.core.gdpr:forget_subjects:270 - GDPR erasure request: strategy=nullify, subjects=1, pii_columns=['email', 'full_name', 'phone', 'date_of_birth'], timestamp=2026-02-21T13:01:21.387882+00:00


After NULLIFY:
shape: (1, 4)
┌─────────────┬───────┬───────────┬───────┐
│ customer_id ┆ email ┆ full_name ┆ phone │
│ ---         ┆ ---   ┆ ---       ┆ ---   │
│ i64         ┆ str   ┆ str       ┆ str   │
╞═════════════╪═══════╪═══════════╪═══════╡
│ 8571        ┆ null  ┆ null      ┆ null  │
└─────────────┴───────┴───────────┴───────┘


In [10]:
# Strategy 2: HASH — PII replaced with SHA-256, references intact
df_hashed = forget_subjects(
    good_df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=[subject_id],
    erasure_strategy="hash",
    hash_salt="my-secret-salt-2026",
)
after_hash = df_hashed.filter(pl.col("customer_id").cast(pl.Utf8) == subject_id)
print("After HASH (PII is deterministically hashed — can still JOIN):")
print(after_hash.select(["customer_id","email","full_name","phone"]))

2026-02-21 13:01:21.405 | INFO     | lakelogic.core.gdpr:forget_subjects:270 - GDPR erasure request: strategy=hash, subjects=1, pii_columns=['email', 'full_name', 'phone', 'date_of_birth'], timestamp=2026-02-21T13:01:21.405151+00:00


After HASH (PII is deterministically hashed — can still JOIN):
shape: (1, 4)
┌─────────────┬────────────────────────────┬───────────────────────────┬───────────────────────────┐
│ customer_id ┆ email                      ┆ full_name                 ┆ phone                     │
│ ---         ┆ ---                        ┆ ---                       ┆ ---                       │
│ i64         ┆ str                        ┆ str                       ┆ str                       │
╞═════════════╪════════════════════════════╪═══════════════════════════╪═══════════════════════════╡
│ 8571        ┆ 6831589f7c1e5dfa55fc314879 ┆ 737b3eb3c4220c5aa43345ba5 ┆ 24fff25c826fba3483e45b9b7 │
│             ┆ 6d63…                      ┆ 4eb35…                    ┆ 80800…                    │
└─────────────┴────────────────────────────┴───────────────────────────┴───────────────────────────┘


In [11]:
# Strategy 3: REDACT — human-readable, best for audit logs
df_redacted = forget_subjects(
    good_df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=[subject_id],
    erasure_strategy="redact",
)
after_redact = df_redacted.filter(pl.col("customer_id").cast(pl.Utf8) == subject_id)
print("After REDACT:")
print(after_redact.select(["customer_id","email","full_name","phone"]))

2026-02-21 13:01:21.435 | INFO     | lakelogic.core.gdpr:forget_subjects:270 - GDPR erasure request: strategy=redact, subjects=1, pii_columns=['email', 'full_name', 'phone', 'date_of_birth'], timestamp=2026-02-21T13:01:21.435691+00:00


After REDACT:
shape: (1, 4)
┌─────────────┬────────────────┬────────────────┬────────────────┐
│ customer_id ┆ email          ┆ full_name      ┆ phone          │
│ ---         ┆ ---            ┆ ---            ┆ ---            │
│ i64         ┆ str            ┆ str            ┆ str            │
╞═════════════╪════════════════╪════════════════╪════════════════╡
│ 8571        ┆ ***REDACTED*** ┆ ***REDACTED*** ┆ ***REDACTED*** │
└─────────────┴────────────────┴────────────────┴────────────────┘


### Generate a compliance-ready audit report

This is what you send to the DPO or attach to a DSAR (Data Subject Access Request) response.

In [12]:
import json

report = generate_erasure_report(
    contract=proc.contract,
    subject_column="customer_id",
    subject_ids=[subject_id],
    erasure_strategy="nullify",
    affected_rows=len(before),
)

print(json.dumps(report, indent=2, default=str))

{
  "report_type": "gdpr_erasure",
  "timestamp": "2026-02-21T13:01:21.452415+00:00",
  "contract": "customers",
  "subject_column": "customer_id",
  "subjects_erased": 1,
  "erasure_strategy": "nullify",
  "pii_columns_affected": [
    "email",
    "full_name",
    "phone",
    "date_of_birth"
  ],
  "affected_rows": 1,
  "compliance_note": "PII data has been erased in accordance with GDPR Article 17 (Right to Erasure). This report should be retained for audit purposes."
}


---

## Step 5 — GDPR: Mask All PII for Dev / Test

You want to give the data engineering team a copy of production data to test with —
but with all PII stripped. `mask_pii_columns()` blanket-anonymises every field
marked `pii: true` in the contract (which we got from `meta: {pii: true}` in dbt).

In [13]:
df_masked = mask_pii_columns(
    good_df,
    proc.contract,
    strategy="redact",  # or "nullify" or "hash"
)

print("PII fields in dev copy:")
pii_preview = ["customer_id","email","full_name","phone","date_of_birth"]
print(df_masked.select([c for c in pii_preview if c in df_masked.columns]).head(5))
print()
print("Non-PII fields are untouched:")
print(df_masked.select(["tier","country","total_spend","is_active"]).head(5))

2026-02-21 13:01:21.462 | INFO     | lakelogic.core.gdpr:mask_pii_columns:338 - Masking PII columns: ['email', 'full_name', 'phone', 'date_of_birth'] with strategy 'redact'
2026-02-21 13:01:21.464 | INFO     | lakelogic.core.gdpr:_mask_polars:197 - PII masking (redact): masked 4 columns across all rows.


PII fields in dev copy:
shape: (5, 5)
┌─────────────┬────────────────┬────────────────┬────────────────┬────────────────┐
│ customer_id ┆ email          ┆ full_name      ┆ phone          ┆ date_of_birth  │
│ ---         ┆ ---            ┆ ---            ┆ ---            ┆ ---            │
│ i64         ┆ str            ┆ str            ┆ str            ┆ str            │
╞═════════════╪════════════════╪════════════════╪════════════════╪════════════════╡
│ 8571        ┆ ***REDACTED*** ┆ ***REDACTED*** ┆ ***REDACTED*** ┆ ***REDACTED*** │
│ 8626        ┆ ***REDACTED*** ┆ ***REDACTED*** ┆ ***REDACTED*** ┆ ***REDACTED*** │
│ 8822        ┆ ***REDACTED*** ┆ ***REDACTED*** ┆ ***REDACTED*** ┆ ***REDACTED*** │
│ 7228        ┆ ***REDACTED*** ┆ ***REDACTED*** ┆ ***REDACTED*** ┆ ***REDACTED*** │
│ 439         ┆ ***REDACTED*** ┆ ***REDACTED*** ┆ ***REDACTED*** ┆ ***REDACTED*** │
└─────────────┴────────────────┴────────────────┴────────────────┴────────────────┘

Non-PII fields are untouched:
shape: 

---

## Step 6 — Batch Erasure + Audit Report

In practice, erasure requests come in batches. One call handles them all.

In [14]:
# Simulate a batch of 5 erasure requests received this week
erasure_ids = [str(good_df["customer_id"][i]) for i in range(5)]
print(f"Batch erasure for {len(erasure_ids)} subjects: {erasure_ids}")

df_batch_erased = forget_subjects(
    good_df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=erasure_ids,
    erasure_strategy="hash",
    hash_salt="weekly-erasure-salt",
)

# Verify erased rows are hashed
erased_sample = df_batch_erased.filter(
    pl.col("customer_id").cast(pl.Utf8).is_in(erasure_ids)
)
print(f"\nErased rows: {len(erased_sample)}")
print(erased_sample.select(["customer_id","email","full_name"]).head(5))

# DPO audit report for the batch
batch_report = generate_erasure_report(
    contract=proc.contract,
    subject_column="customer_id",
    subject_ids=erasure_ids,
    erasure_strategy="hash",
    affected_rows=len(erased_sample),
)
print()
import json
print(json.dumps(batch_report, indent=2, default=str))

2026-02-21 13:01:21.486 | INFO     | lakelogic.core.gdpr:forget_subjects:270 - GDPR erasure request: strategy=hash, subjects=5, pii_columns=['email', 'full_name', 'phone', 'date_of_birth'], timestamp=2026-02-21T13:01:21.486070+00:00


Batch erasure for 5 subjects: ['8571', '8626', '8822', '7228', '439']

Erased rows: 5
shape: (5, 3)
┌─────────────┬─────────────────────────────────┬─────────────────────────────────┐
│ customer_id ┆ email                           ┆ full_name                       │
│ ---         ┆ ---                             ┆ ---                             │
│ i64         ┆ str                             ┆ str                             │
╞═════════════╪═════════════════════════════════╪═════════════════════════════════╡
│ 8571        ┆ 0484a10e87add9fe33242f04cc7842… ┆ 3a34c5e8bd0b5886bc8e40b34cc96a… │
│ 8626        ┆ 81eb260c1ad07032441cf69148fff1… ┆ 3152292eb0a1850831261928a3f0d3… │
│ 8822        ┆ 1351efb2dbb6c78af4b85a50a433e4… ┆ 76a86a766952d733512e51f3390f55… │
│ 7228        ┆ 4773ba100e14c072764ed9eab6a554… ┆ 0eebfe1dbe27aa41d57b2a2306176a… │
│ 439         ┆ 51ffece5bdf828edd16df2967f00bb… ┆ 5e31409437c4f05ce26967052a5e8f… │
└─────────────┴─────────────────────────────────┴───────────

---

## Step 7 — CLI Reference

Everything above is also available from the terminal — no notebook required.

In [ ]:
schema_path = str(SCHEMA_FILE)

print("# ── Import dbt schema → LakeLogic contract ─────────────────────────")
print(f"lakelogic import-dbt --schema {schema_path} --model customers --output contracts/")
print()
print("# ── Dry-run preview (no files written) ──────────────────────────────")
print(f"lakelogic import-dbt --schema {schema_path} --model customers --dry-run")
print()
print("# ── Import every model in the schema file ────────────────────────────")
print(f"lakelogic import-dbt --schema {schema_path} --output contracts/")
print()
print("# ── Run validation against a data file ──────────────────────────────")
print(f"lakelogic run --contract contracts/customers.yaml --source data/customers.csv")
print()
print("# ── Generate 500 synthetic test rows ─────────────────────────────────")
print(f"lakelogic generate --contract contracts/customers.yaml --rows 500 --invalid-ratio 0.1 --output data/test_customers.parquet")

# ── Import dbt schema → LakeLogic contract ─────────────────────────
lakelogic import-dbt --schema d:\Github\_SaaS\lakelogic\examples\01_quickstart\dbt_schema.yml --model customers --output contracts/

# ── Dry-run preview (no files written) ──────────────────────────────
lakelogic import-dbt --schema d:\Github\_SaaS\lakelogic\examples\01_quickstart\dbt_schema.yml --model customers --dry-run

# ── Import every model in the schema file ────────────────────────────
lakelogic import-dbt --schema d:\Github\_SaaS\lakelogic\examples\01_quickstart\dbt_schema.yml --output contracts/

# ── Run validation against a data file ──────────────────────────────
lakelogic run --contract contracts/customers.yaml --source data/customers.csv

# ── Generate 500 synthetic test rows ─────────────────────────────────
lakelogic generate --contract contracts/customers.yaml --rows 500 --invalid-ratio 0.1 --output data/test_customers.parquet


: 

---

## Summary

```
dbt_schema.yml          ← the ONLY schema file you maintain
         │
         ├─ DbtAdapter()              → DataContract (PII, required, rules all mapped)
         │
         ├─ DataGenerator.from_dbt()  → synthetic test data  (Step 2)
         ├─ DataProcessor.from_dbt()  → quality validation    (Step 3)
         ├─ forget_subjects()         → GDPR right-to-erasure (Step 4)
         ├─ mask_pii_columns()        → dev/test PII masking  (Step 5)
         └─ generate_erasure_report() → DPO audit trail       (Steps 4 & 6)
```

**One schema file → zero duplication → full compliance.**

### Next steps

- **`02_core_patterns`** — Medallion architecture and reference joins
- **`06_advanced_workflows/polars_streaming`** — Process 50 GB files without OOM
- **`06_advanced_workflows/gdpr_compliance`** — Full GDPR demo with audit reports
- **`06_advanced_workflows/shared_governance_scale`** — 26 contracts in parallel with shared templates
